# 1. Read the point cloud captured using ros topic from semantic lidar to get semantic data

In [3]:
import pandas as pd

df_road_person = pd.read_csv("/home/nero/Documents/person and road for semantic analysis.csv")
df_road_person

,//X,Y,Z,ObjIdx,ObjTag
0,-2.44912,7.664550,0.280984,0.0,5.0
1,-2.40729,7.695080,0.281560,0.0,5.0
2,7.16648,-8.060780,0.376651,415.0,5.0
3,7.10137,-8.086210,0.375811,415.0,5.0
4,7.13555,-8.225700,0.380266,415.0,18.0
...,...,...,...,...,...
29568,-5.19533,-0.158345,-2.401680,0.0,7.0
29569,-5.19619,-0.126680,-2.401680,0.0,7.0
29570,-5.19689,-0.095015,-2.401680,0.0,7.0
29571,-5.19736,-0.063345,-2.401680,0.0,7.0


In [4]:
df_road_person["ObjIdx"].unique()

array([  0., 415., 580.])

## O label is for road or fixed object, 580 label is for person, 415 label maybe for poles or some traffic light

# 2. Seperate person and road from the cloud and test using CloudCompare by visualizing

## Reseting the Index is important! When concating two dataframe, it will create problems so better reset it now.

In [22]:
df_person = df_road_person[df_road_person["ObjIdx"] == 580].reset_index(drop=True)
df_person

,//X,Y,Z,ObjIdx,ObjTag
0,4.05199,4.03352,-0.523512,580.0,4.0
1,4.06547,3.99792,-0.522097,580.0,4.0
2,4.09164,3.97493,-0.522343,580.0,4.0
3,4.13305,3.96648,-0.524532,580.0,4.0
4,4.00406,4.03471,-0.563076,580.0,4.0
...,...,...,...,...,...
531,3.91473,3.89692,-2.355880,580.0,4.0
532,3.92980,3.86452,-2.350730,580.0,4.0
533,3.95355,3.84078,-2.350900,580.0,4.0
534,3.98633,3.82566,-2.356480,580.0,4.0


In [23]:
df_person.to_csv("csv/latest_person_only.csv", index=None)

In [24]:
df_road = df_road_person[df_road_person["ObjIdx"] == 0].reset_index(drop=True)
df_road.to_csv("csv/latest_road_only.csv", index=None)

# 3. Transformation of person object(a simple rotation around the origin will work for testing)

## we rotate the person around the origin clockwise 90 degrees. we need to pass N x 2 array to our rotatematrix. Since we will not rotate in Z direction, its okay to rotate only using X and Y

In [25]:
array_person_xy_unrotated = df_person.iloc[:, :2].to_numpy()

In [26]:
import numpy as np

def rotateMatrix(a):
    return np.array([[np.cos(a), -np.sin(a)], [np.sin(a), np.cos(a)]])
    
angle_in_degrees = -90 # rotation in clockwise
angle_in_rads = np.deg2rad(angle_in_degrees)
array_person_xy_rotated = array_person_xy_unrotated @ rotateMatrix(angle_in_rads).T

# 4. After the person array has been rotated, we can save it and visulize in CloudCompare to see the change

In [29]:
df_person_xy_rotated = pd.DataFrame(array_person_xy_rotated, columns=["X", "Y",])
df_person_xy_rotated.head()

,X,Y
0,4.03352,-4.05199
1,3.99792,-4.06547
2,3.97493,-4.09164
3,3.96648,-4.13305
4,4.03471,-4.00406


In [30]:
df_person_remaining_data = df_person.iloc[:, 2:]
df_person_remaining_data

,Z,ObjIdx,ObjTag
0,-0.523512,580.0,4.0
1,-0.522097,580.0,4.0
2,-0.522343,580.0,4.0
3,-0.524532,580.0,4.0
4,-0.563076,580.0,4.0
...,...,...,...
531,-2.355880,580.0,4.0
532,-2.350730,580.0,4.0
533,-2.350900,580.0,4.0
534,-2.356480,580.0,4.0


### Merging the person data (x_rotated, y_rotated, z, ObjIdx, ObjTag)

In [31]:
df_person_rotated = pd.concat([df_person_xy_rotated, df_person_remaining_data], axis=1,)
df_person_rotated

,X,Y,Z,ObjIdx,ObjTag
0,4.03352,-4.05199,-0.523512,580.0,4.0
1,3.99792,-4.06547,-0.522097,580.0,4.0
2,3.97493,-4.09164,-0.522343,580.0,4.0
3,3.96648,-4.13305,-0.524532,580.0,4.0
4,4.03471,-4.00406,-0.563076,580.0,4.0
...,...,...,...,...,...
531,3.89692,-3.91473,-2.355880,580.0,4.0
532,3.86452,-3.92980,-2.350730,580.0,4.0
533,3.84078,-3.95355,-2.350900,580.0,4.0
534,3.82566,-3.98633,-2.356480,580.0,4.0


### Saving the rotated person data to visualize

In [32]:
df_person_rotated.to_csv("csv/latest_rotated_person.csv", index=None)

# 5. Rotated Person Surface Reconstruction as a Mesh using Open3d

In [33]:
df_person = pd.read_csv("csv/latest_rotated_person.csv")
df_person

,X,Y,Z,ObjIdx,ObjTag
0,4.03352,-4.05199,-0.523512,580.0,4.0
1,3.99792,-4.06547,-0.522097,580.0,4.0
2,3.97493,-4.09164,-0.522343,580.0,4.0
3,3.96648,-4.13305,-0.524532,580.0,4.0
4,4.03471,-4.00406,-0.563076,580.0,4.0
...,...,...,...,...,...
531,3.89692,-3.91473,-2.355880,580.0,4.0
532,3.86452,-3.92980,-2.350730,580.0,4.0
533,3.84078,-3.95355,-2.350900,580.0,4.0
534,3.82566,-3.98633,-2.356480,580.0,4.0


In [34]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df_person.iloc[:,:3].to_numpy()) # Need to pass numpy array

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [35]:
o3d.visualization.draw_geometries([pcd],)

### Surface reconstruction needs point cloud with normal, so we need to calculate appropriate Normals.

### Calculating Normal is an important step, Better surface reconstruction depends on the quality of Normals

In [36]:
search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.37, max_nn=6)
pcd.estimate_normals(search_param=search_param)

### We use Poisson Surface Reconstruction because it is better than other two options in open3d

In [37]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
mesh.compute_vertex_normals()
# Paint it gray. Not necessary but the reflection of lighting is hardly perceivable with black surfaces.
mesh.paint_uniform_color(np.array([[0.5],[0.5],[0.5]]))
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 536 / 536
[Open3D DEBUG] #   Got kernel density: 0.03086400032043457 (s), 319.6328125 (MB) / 319.6328125 (MB) / 319 (MB)
[Open3D DEBUG] #     Got normal field: 0.024096012115478516 (s), 320.1796875 (MB) / 320.1796875 (MB) / 320 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 8.935923e-04 / 4.789655e-01
[Open3D DEBUG] #       Finalized tree: 0.07201695442199707 (s), 323.30078125 (MB) / 323.30078125 (MB) / 323 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.10940694808959961 (s), 323.30078125 (MB) / 323.30078125 (MB) / 323 (MB)
[Open3D DEBUG] #Set point constraints: 0.0023348331451416016 (s), 323.30078125 (MB) / 323.30078125 (MB) / 323 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 89111 / 71848 / 29993
[Open3D DEBUG] Memory Usage: 323.301 MB
[Open3D DEBUG] # Linear system solved: 0.10134387016296387 (s), 326.25390625 (MB) / 326.25390625 (MB) / 326 (MB)
[Open3D DEBUG] Got average: 0.00150203704833984

### Visualizing the densities. As a next step we can use this densities value to only take the person mesh

In [39]:
import matplotlib.pyplot as plt

print('visualize densities')
densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh])

visualize densities


### Filtering out densities based on certain threshold. We need to experiment more to get an appropriate value.

In [40]:
print('remove low density vertices')
vertices_to_remove = densities < np.quantile(densities, 0.5)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)

mesh.compute_vertex_normals()
# Paint it gray. Not necessary but the reflection of lighting is hardly perceivable with black surfaces.
mesh.paint_uniform_color(np.array([[0.5],[0.5],[0.5]]))

o3d.visualization.draw_geometries([mesh],)

remove low density vertices
TriangleMesh with 1756 points and 3364 triangles.


In [41]:
# Save the person pcd and reconstructed surface of person for future use
o3d.io.write_point_cloud("csv/latest_reconstructed_person_pcd.ply", pcd)
o3d.io.write_triangle_mesh("csv/latest_reconstructed_person_mesh.obj", mesh)

[Open3D WARNING] Write OBJ can not include triangle normals.


True

### Now that we have mesh of person, we can use RayCasting to get point cloud

# 6.RayCasting to get intersecting point of rays on the Reconstructed Surface of Person to create pointcloud

### We need to create rays for raycasting using open3d library. We need to specify the direction and origin of ray

### Origin of Ray can be the origin of the pointcloud (0,0,0)

### For the Direction, we choose the point cloud point values on the target cloud. As these points are also direction of the original rays

### We might also calculate a matrix of directions using camera features but this steps seems more practical and we donot need to downsample the pointcloud later

In [42]:
# using the ea

df_road = pd.read_csv("csv/latest_road_only.csv")
df_road

,//X,Y,Z,ObjIdx,ObjTag
0,-2.44912,7.664550,0.280984,0.0,5.0
1,-2.40729,7.695080,0.281560,0.0,5.0
2,8.54879,-14.135900,0.576888,0.0,5.0
3,-2.44904,7.664230,0.221176,0.0,5.0
4,-2.40713,7.694510,0.221623,0.0,5.0
...,...,...,...,...,...
28936,-5.19533,-0.158345,-2.401680,0.0,7.0
28937,-5.19619,-0.126680,-2.401680,0.0,7.0
28938,-5.19689,-0.095015,-2.401680,0.0,7.0
28939,-5.19736,-0.063345,-2.401680,0.0,7.0


In [43]:
df_ray_origin = pd.DataFrame(np.zeros((df_road.shape[0], 3)))
df_ray_origin

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
28936,0.0,0.0,0.0
28937,0.0,0.0,0.0
28938,0.0,0.0,0.0
28939,0.0,0.0,0.0


In [44]:
df_ray_origin_with_target_direction = pd.concat([df_ray_origin, df_road.iloc[:, :3]], axis=1)
df_ray_origin_with_target_direction

,0,1,2,//X,Y,Z
0,0.0,0.0,0.0,-2.44912,7.664550,0.280984
1,0.0,0.0,0.0,-2.40729,7.695080,0.281560
2,0.0,0.0,0.0,8.54879,-14.135900,0.576888
3,0.0,0.0,0.0,-2.44904,7.664230,0.221176
4,0.0,0.0,0.0,-2.40713,7.694510,0.221623
...,...,...,...,...,...,...
28936,0.0,0.0,0.0,-5.19533,-0.158345,-2.401680
28937,0.0,0.0,0.0,-5.19619,-0.126680,-2.401680
28938,0.0,0.0,0.0,-5.19689,-0.095015,-2.401680
28939,0.0,0.0,0.0,-5.19736,-0.063345,-2.401680


In [45]:
rays_array = df_ray_origin_with_target_direction.to_numpy()
rays_array

array([[  0.        ,   0.        ,   0.        ,  -2.44912004,
          7.66454983,   0.28098401],
       [  0.        ,   0.        ,   0.        ,  -2.40728998,
          7.6950798 ,   0.28156   ],
       [  0.        ,   0.        ,   0.        ,   8.54878998,
        -14.13589954,   0.57688802],
       ...,
       [  0.        ,   0.        ,   0.        ,  -5.19688988,
         -0.0950146 ,  -2.40167999],
       [  0.        ,   0.        ,   0.        ,  -5.19736004,
         -0.0633447 ,  -2.40167999],
       [  0.        ,   0.        ,   0.        ,  -5.19765997,
         -0.0316699 ,  -2.40167999]])

### We have now a group of rays. We need to figure out the intersection point on the person mesh

In [47]:
# Create a scene and add the traingle mesh of the object
mesh_new = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh_new) # we get typeerror if we directly pass the mesh


0

In [48]:
# Creating Rays for casting using the rays array created earlier
rays = o3d.core.Tensor(rays_array,dtype=o3d.core.Dtype.Float32)

# Casting rays on the scene
ans = scene.cast_rays(rays)

In [53]:
print(ans.keys())

dict_keys(['primitive_uvs', 'primitive_ids', 'geometry_ids', 'primitive_normals', 't_hit'])


### The ray is passed through the scene. We can now see on which traingles are the ray intersected

In [98]:
triangle_ids = [i.numpy() for i in ans["primitive_ids"] if i != scene.INVALID_ID]

In [99]:
len(triangle_ids)

488

### Triangle Ids contains ids of triangle in the mesh through which the ray passed

In [100]:
triangle_indices = [i.numpy() for i in mesh_new.triangle.indices] # list of all the triangles in the mesh

In [101]:
len(triangle_indices)

3364

In [102]:
triangle_indices[0]

array([ 0,  1, 20], dtype=int64)

### Each value in triangle indices represents the three vertex  index of triangle.

### We can use the vertex index and get the corresponding vertex value in x,y,z coordinate

In [103]:
vertex_list = [i.numpy() for i in mesh_new.vertex.positions]

In [124]:
all_intersected_points = np.zeros((len(triangle_ids), 3)) # Creating an empty array
for indx, triangle_id in enumerate(triangle_ids): # List of Triangles through which rays intersected
    triangle_vertex = triangle_indices[triangle_id] # Gives the list of vertex
    vertex0 = vertex_list[triangle_vertex[0]] # Gives the corresponding vertex value in x,y,z
    vertex1 = vertex_list[triangle_vertex[1]] # Gives the corresponding vertex value in x,y,z
    vertex2 = vertex_list[triangle_vertex[2]] # Gives the corresponding vertex value in x,y,z
    all_intersected_points[indx] = [(vertex0[i] + vertex1[i] + vertex2[i])/3 for i in range(3)] # Computing the centroid for now
print(all_intersected_points)

[[ 4.25822608 -3.87941424 -0.90936716]
 [ 4.2051007  -3.88251813 -0.91812126]
 [ 4.25822608 -3.85681725 -0.97124648]
 ...
 [ 3.83029366 -4.05920855 -2.39500237]
 [ 3.88194116 -3.92918523 -2.39296627]
 [ 3.83168411 -4.01002661 -2.41776403]]


In [125]:
df_person_pcd_after_raycast = pd.DataFrame(all_intersected_points)
df_person_pcd_after_raycast

,0,1,2
0,4.258226,-3.879414,-0.909367
1,4.205101,-3.882518,-0.918121
2,4.258226,-3.856817,-0.971246
3,4.236976,-3.871621,-0.949996
4,4.194476,-3.880468,-0.949996
...,...,...,...
483,3.842904,-3.962596,-2.352502
484,3.829991,-3.995805,-2.363127
485,3.830294,-4.059209,-2.395002
486,3.881941,-3.929185,-2.392966


# 7. Saving the new pointcloud of person after Raycasting

In [126]:
df_person_pcd_after_raycast.to_csv("csv/latest_person_pcd_after_raycast.csv", index=False)

### using open3d to load and visualize the person

In [127]:
df_person_after_raycast = pd.read_csv("csv/latest_person_pcd_after_raycast.csv")
df_person_after_raycast

,0,1,2
0,4.258226,-3.879414,-0.909367
1,4.205101,-3.882518,-0.918121
2,4.258226,-3.856817,-0.971246
3,4.236976,-3.871621,-0.949996
4,4.194476,-3.880468,-0.949996
...,...,...,...
483,3.842904,-3.962596,-2.352502
484,3.829991,-3.995805,-2.363127
485,3.830294,-4.059209,-2.395002
486,3.881941,-3.929185,-2.392966


In [128]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df_person_after_raycast.to_numpy()) # Need to pass numpy array

o3d.visualization.draw_geometries([pcd],)


# 8. Shadow Casting using Hidden Point Removal

### First step concatenate the new person pcd after raycast and road pcd

In [136]:
df_road = pd.read_csv("csv/latest_road_only.csv").iloc[:, :3]
df_road.columns = [0, 1, 2]
df_road

,0,1,2
0,-2.44912,7.664550,0.280984
1,-2.40729,7.695080,0.281560
2,8.54879,-14.135900,0.576888
3,-2.44904,7.664230,0.221176
4,-2.40713,7.694510,0.221623
...,...,...,...
28936,-5.19533,-0.158345,-2.401680
28937,-5.19619,-0.126680,-2.401680
28938,-5.19689,-0.095015,-2.401680
28939,-5.19736,-0.063345,-2.401680


In [148]:
df_person_new = pd.read_csv("csv/latest_person_pcd_after_raycast.csv")
df_person_new.columns = [0,1,2]
df_person_new

,0,1,2
0,4.258226,-3.879414,-0.909367
1,4.205101,-3.882518,-0.918121
2,4.258226,-3.856817,-0.971246
3,4.236976,-3.871621,-0.949996
4,4.194476,-3.880468,-0.949996
...,...,...,...
483,3.842904,-3.962596,-2.352502
484,3.829991,-3.995805,-2.363127
485,3.830294,-4.059209,-2.395002
486,3.881941,-3.929185,-2.392966


In [149]:
df_person_new.columns

Int64Index([0, 1, 2], dtype='int64')

In [150]:
df_road.columns

Int64Index([0, 1, 2], dtype='int64')

In [151]:
df_road_person_new = pd.concat([df_road, df_person_new], axis=0)
df_road_person_new

,0,1,2
0,-2.449120,7.664550,0.280984
1,-2.407290,7.695080,0.281560
2,8.548790,-14.135900,0.576888
3,-2.449040,7.664230,0.221176
4,-2.407130,7.694510,0.221623
...,...,...,...
483,3.842904,-3.962596,-2.352502
484,3.829991,-3.995805,-2.363127
485,3.830294,-4.059209,-2.395002
486,3.881941,-3.929185,-2.392966


### creating point cloud from the dataframe

In [156]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df_road_person_new.to_numpy()) # Need to pass numpy array
o3d.visualization.draw_geometries([pcd],)

### We have some findigs when visualizing. When visualizing the new raycasted pcd of person, I could see that the head part of person is not much available or missing.

### This is because we have taken a part of the point cloud of road. So rays farther away from the origin is not present and hence no rays created that passes through the head of the person

### Hidden Point Removal Used for Shadow Casting

In [157]:
diameter = np.linalg.norm(
    np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
o3d.visualization.draw_geometries([pcd])

print("Define parameters used for hidden_point_removal")
camera = o3d.core.Tensor([0, 0, 0], o3d.core.float32)
radius = diameter * 300

print("Get all points that are visible from given view point")
pcd = o3d.t.geometry.PointCloud.from_legacy(pcd)
_, pt_map = pcd.hidden_point_removal(camera, radius)
pcd = pcd.select_by_index(pt_map)

print("Visualize result")
o3d.visualization.draw_geometries([pcd.to_legacy()])

Define parameters used for hidden_point_removal
Get all points that are visible from given view point
Visualize result


### Saving the pcd after shadow casting for future visualization

In [178]:
o3d.io.write_point_cloud("csv/latest_shadow_casted_from_raycasted_person_on_road.ply", pcd.to_legacy())

True

# 9. Next Steps

## i. Need to get a proper reconstructed surface

## ii. Proper Shadow Calculation with good params maybe some error metrices

## iii. Pipelining the process

## iv. Need to think about the proper boundaries before ray casting as the target pcd boundary is important in determining rays direction

## v. Transformation of object to another point and perform PCD ICP Registration

## vi. Maybe Create a Proper Visualization tools to do these things interactively